In [1]:
using Revise, Pkg
Pkg.activate("/home/louise/MSA/BpAlignGpu.jl")
using BpAlignGpu

  Activating project at `~/MSA/BpAlignGpu.jl`


In [2]:
q=21
ctype=Symbol("amino")
typel=Symbol("bm")
T = Float32
muext = 0.50;
muint = 2.50;

In [3]:
damp=T(0.0)
tol=T(1e-5)
tolnorm=T(1e-5)
tmax=10
upscheme=:sequential # :random or :sequential
lr=:sce  # :sce or :mf 
beta=T(1.0)
verbose=true
pa = ParamAlgo(damp, tol, tolnorm, tmax, upscheme, lr, beta, verbose)

ParamAlgo{Float32}
-------------
damp=0.0
tol=1.0e-5
tolnorm=1.0e-5
tmax=10
upscheme=sequential
lr=sce
beta=1.0
verbose=true
-------------


In [4]:
(N, L) = (5, 5) 

using Random
header = "myseq"
myseq = randstring('A':'Z', N)
seq = Seq(header, myseq, ctype)

lambda_o = ones(L)
lambda_e = ones(L)
H = rand(q,L)
J = rand(q,q,L,L)
J = J .+ permutedims(J, (2,1,4,3));

In [5]:
pm = ParamModel{T}(N, L, q, muint, muext, lambda_o, lambda_e, H, J)

ParamModel{Float32}[L=5 N=5 q=21]


In [6]:
bpm = BPMessages(seq, pm)
bpb = BPBeliefs(N, L)
lrf = LongRangeFields(N, L)
af = AllFields(bpm, bpb, lrf)

┌ Warning: The NVIDIA driver on this system only supports up to CUDA 11.0.0.
│ For performance reasons, it is recommended to upgrade to a driver that supports CUDA 11.2 or higher.
└ @ CUDA /home/louise/.julia/packages/CUDA/bki2w/src/initialization.jl:70


AllFields{Float32}[L=5 N=5 ongpu=true]

In [7]:
BpAlignGpu.update_F!(af, pm, pa)
BpAlignGpu.update_hF!(af, pm, pa)
BpAlignGpu.update_B!(af, pm, pa)
BpAlignGpu.update_hB!(af, pm, pa)
BpAlignGpu.update_beliefs!(af, pm, pa)
BpAlignGpu.update_jointchain!(af, pm, pa)
BpAlignGpu.update_conditional_chain!(af, pa)
BpAlignGpu.update_conditional_all!(af, pm)
BpAlignGpu.update_f!(af)
BpAlignGpu.update_gold!(af)


In [22]:
conditional = af.bpb.conditional;

In [23]:
using Tullio
np1 = size(conditional, 1)

7

In [60]:
mask = similar(conditional)
@tullio mask[ni, xi, nj, xj, i, j] = (i < j) (ni in 1:np1, xi in 1:2, nj in 1:np1, xj in 1:2, i in 1:L, j in 1:L)
mask = reshape(permutedims(mask, (1, 2, 5, 3, 4, 6)), L * 2 * np1, L * 2 * np1)
J = reshape(permutedims(Jseq, (1, 2, 5, 3, 4, 6)), L * 2 * np1, L * 2 * np1)
cond = reshape(permutedims(conditional, (1, 2, 5, 3, 4, 6)), L * 2 * np1, L * 2 * np1)

In [123]:
mask2 = similar(conditional)
@tullio mask2[ni, xi, nj, xj, i, j] = (i <= j) (ni in 1:np1, xi in 1:2, nj in 1:np1, xj in 1:2, i in 1:L, j in 1:L);
mask2 = reshape(permutedims(mask2, (1, 2, 5, 3, 4, 6)), L * 2 * np1, L * 2 * np1);


In [117]:
ff = fill(NaN, N+2, 2, L)
ff .= reshape(diag(- (mask .* cond') * (JJ * (cond .* mask))), np1, 2, L);
af.lrf.f[:,:,4] == ff[:,:,4]

true

In [120]:
using LinearAlgebra

In [125]:
tmp =  (mask2 .* cond') * (JJ * (cond .* mask));

In [126]:
tmp = tmp .* mask2;

In [127]:
af.lrf.g[:,1,:,1,1]

7×7 CUDA.CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 2.93025  2.88094  3.06702  2.52889  2.63884  0.0  2.44922
 2.93025  2.88094  3.06702  2.52889  2.63884  0.0  2.44922
 2.93025  2.88094  3.06702  2.52889  2.63884  0.0  2.44922
 2.93025  2.88094  3.06702  2.52889  2.63884  0.0  2.44922
 2.93025  2.88094  3.06702  2.52889  2.63884  0.0  2.44922
 2.93025  2.88094  3.06702  2.52889  2.63884  0.0  2.44922
 2.93025  2.88094  3.06702  2.52889  2.63884  0.0  2.44922

In [113]:
range0 = 1:N+2
range2 = N+3:2N+4

8:14

In [128]:
l=2
tt = tmp[1+(l-1)*(2N+4):l*(2N+4), 1+l*(2N+4):(l+1)*(2N+4)]
tt

14×14 CUDA.CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 7.49353  7.15982  7.47201  8.68782  …  7.41762  8.43014  7.83517  0.0
 7.42065  6.99903  7.26551  8.43632     7.29301  8.20321  7.94613  0.0
 7.43419  7.33677  6.76069  8.59615     7.28562  8.32463  8.15603  0.0
 6.42441  6.83712  6.52164  8.20895     6.62386  7.87454  7.21502  0.0
 6.85019  7.16905  6.707    8.55097     6.9749   8.21667  7.73328  0.0
 1.22094  1.70717  1.55058  2.55824  …  1.47236  2.32803  1.78468  0.0
 6.74147  6.82414  7.2086   8.83882     6.90965  8.44434  7.44801  0.0
 0.0      0.0      0.0      0.0         0.0      0.0      0.0      0.0
 7.67694  7.19383  7.13279  7.71525     7.40733  7.65853  7.53682  0.0
 6.82609  6.7404   8.12257  7.95443     7.12566  7.78665  8.09158  0.0
 8.46455  7.98247  7.74686  8.22817  …  8.15918  8.21719  7.68975  0.0
 7.75525  7.2978   7.45939  8.03287     7.56123  7.94234  7.99443  0.0
 8.65305  7.98309  8.17111  8.62511     8.33664  8.57505  7.83665  0.0
 0.0      0.0      0.0

In [129]:
gg = fill(NaN, N+2, 2, N+2, 2, L);

In [130]:
for l=1:L-1
    gg[:,:,:,:,l] .= reshape(tmp[1+(l-1)*(2N+4):l*(2N+4), 1+l*(2N+4):(l+1)*(2N+4)], np1, 2, np1, 2)
end

In [131]:
gg[:,2,:,2,1]

7×7 Matrix{Float64}:
 0.0  0.0      0.0      0.0      0.0      0.0      0.0
 0.0  4.91901  5.04411  5.28238  5.4391   4.746    0.0
 0.0  5.14566  5.66111  5.45678  5.38388  4.8344   0.0
 0.0  5.043    5.46651  5.37299  5.01412  4.87975  0.0
 0.0  4.37978  4.74074  4.51249  4.92527  4.43412  0.0
 0.0  4.24948  5.34418  4.97769  4.64405  4.3504   0.0
 0.0  0.0      0.0      0.0      0.0      0.0      0.0

In [132]:
af.lrf.g[:,2,:,2,1]

7×7 CUDA.CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0  0.0      0.0      0.0      0.0      0.0      0.0
 0.0  3.55507  3.46071  3.15974  3.49881  3.22212  0.0
 0.0  3.5333   3.37356  3.29429  3.77177  3.0631   0.0
 0.0  3.26476  3.69038  3.65012  3.67949  3.94221  0.0
 0.0  2.91838  2.72676  2.9163   2.49471  2.09051  0.0
 0.0  3.26476  3.69038  3.65012  3.67949  3.94221  0.0
 0.0  0.0      0.0      0.0      0.0      0.0      0.0

In [15]:
#BpAlignGpu.one_bp_sweep!(af, pm, pa)

In [16]:
#BpAlignGpu.test_sweep!(100,af,pm,pa)